# PyMVPA

[PyMVPA](http://www.pymvpa.org/) is a Python package intended to ease statistical learning analyses of large datasets. It offers an extensible framework with a high-level interface to a broad range of algorithms for classification, regression, feature selection, data import and export.

The power in PyMVPA lies in it's flexibility with classifier. PyMVPA is able to use many classifiers from LIBSVM and scikit-learn, and the overall list that are at your hands is impressive. The following are only some of the classifiers that you can chose from:

- Bayesian Linear Regression (BLR)
- Elastic-Net (ENET) regression classifier
- Gaussian Discriminant Analyses (LDA and QDA)
- Gaussian Naive Bayes Classifier (GNB)
- Gaussian Process Regression (GPR)
- GLM-Net (GLMNET) regression and classifier
- k-Nearest-Neighbour classifier (kNN)
- Least angle regression (LARS)
- Penalized logistic regression classifier
- Similarity functions for prototype-based projection
- Sparse Multinomial Logistic Regression classifier (SMLR)
- SVM and SVR machines

**Note:** The content of this notebook is taken and adapted from the [PyMVPA](http://www.pymvpa.org/) homepage and serves an illustrative purpose. For more information and better understanding, go directly to [PyMVPA](http://www.pymvpa.org/).

Having said so, let's take a look at PyMVPA's **Searchlight** example.

## Tutorial Dataset

To do anything in this tutoroial, we first need to download some tutorial data. This can be done with the following command:

In [ ]:
%%bash
curl http://data.pymvpa.org/datasets/tutorial_data/tutorial_data-0.4.tar.gz | tar xvz \
    -C /home/neuro/

In [ ]:
mv /home/neuro/tutorial_data/data /home/neuro/tutorial_data/haxby2001

## Searchlight on fMRI data

The original idea of a spatial searchlight algorithm stems from a paper by [Kriegeskorte et al. (2006)](http://www.pymvpa.org/references.html#kgb06), and has subsequently been used in a number of studies. The most common use for a searchlight is to compute a full cross-validation analysis in each spherical region of interest (ROI) in the brain. This analysis yields a map of (typically) classification accuracies that are often interpreted or post-processed similar to a GLM statistics output map (e.g. subsequent analysis with inferential statistics).

### Preparation

In [ ]:
# Import relevant modules
from mvpa2.suite import *
import os
%matplotlib inline

As searchlight analyses are usually quite expensive in terms of computational resources, we are going to enable some progress output to entertain us while we are waiting.

In [ ]:
# enable debug output for searchlight call
if __debug__:
    debug.active += ["SLC"]

### Get data

In [ ]:
from mvpa2.tutorial_suite import fmri_dataset

In [ ]:
func = '/home/neuro/notebooks/data/dataset_ML.nii.gz'

In [ ]:
from nilearn.image import resample_to_img, math_img
from scipy.ndimage import binary_dilation

def get_mask(mask_type):
    
    # Specify location of the brain and eye image
    brain = '/templates/MNI152_T1_1mm_brain.nii.gz'
    eyes = '/templates/MNI152_T1_1mm_eye.nii.gz'

    # Load region of interest
    if mask_type == 'brain':
        img_resampled = resample_to_img(brain, func)
    elif mask_type == 'eyes':
        img_resampled = resample_to_img(eyes, func)
    elif mask_type == 'both':
        img_roi = math_img("img1 + img2", img1=brain, img2=eyes)
        img_resampled = resample_to_img(img_roi, func)

    # Binarize ROI template
    data_binary = np.array(img_resampled.get_fdata()>=10, dtype=np.int8)

    # Dilate binary mask once
    data_dilated = binary_dilation(data_binary, iterations=1).astype(np.int8)

    # Save binary mask in NIfTI image
    mask = nb.Nifti1Image(data_dilated, img_resampled.affine, img_resampled.header)
    mask.set_data_dtype('i1')
    
    return mask

In [ ]:
mask = get_mask('both')

In [ ]:
mask.to_filename('bla.nii.gz')

In [ ]:
labels = '/home/neuro/notebooks/data/labels.txt'
attrs = np.recfromcsv(labels, delimiter=" ")
stimuli, runs = attrs['labels'], attrs['chunks']

In [ ]:
ds = fmri_dataset(samples=func,
                  targets=[str(s)[2:-1] for s in stimuli],
                  chunks=runs,
                  mask='bla.nii.gz')

del ds.sa['time_coords']
del ds.sa['time_indices']

In [ ]:
ds = ds.copy(deep=False,
             sa=['targets', 'chunks'],
             fa=['voxel_indices'],
             a=['mapper'])

In [ ]:
#!/usr/bin/python
import numpy as np
import os
from os.path import join as opj
from mvpa2.base.hdf5 import h5load, h5save
from mvpa2.clfs.svm import LinearCSVMC, LinearNuSVMC
from mvpa2.clfs.smlr import SMLR
from mvpa2.generators.partition import NFoldPartitioner
from mvpa2.measures.base import CrossValidation
from mvpa2.measures.searchlight import sphere_searchlight
from mvpa2.misc.errorfx import mean_match_accuracy
from mvpa2.mappers.fx import mean_sample
from mvpa2.suite import map2nifti, time

In [ ]:
def fill_in_scattered_results(sl, dataset, roi_ids, results):
    """Function to aggregate results - This requires the searchlight
    conditional attribute 'roi_feature_ids' to be enabled"""
    import numpy as np
    from mvpa2.datasets import Dataset
    resmap = None
    for resblock in results:
        for res in resblock:
            if resmap is None:
                # prepare the result container
                resmap = np.zeros((len(res), dataset.nfeatures),
                                  dtype=res.samples.dtype)
                observ_counter = np.zeros(dataset.nfeatures, dtype=int)
            # project the result onto all features -- love broadcasting!
            resmap[:, res.a.roi_feature_ids] += res.samples
            # increment observation counter for all relevant features
            observ_counter[res.a.roi_feature_ids] += 1
    # when all results have been added up average them according to the number
    # of observations
    observ_mask = observ_counter > 0
    resmap[:, observ_mask] /= observ_counter[observ_mask]
    result_ds = Dataset(resmap,
                        fa={'observations': observ_counter})
    if 'mapper' in dataset.a:
        import copy
        result_ds.a['mapper'] = copy.copy(dataset.a.mapper)
    return result_ds

In [ ]:
# specify balancer and partitioner
partitioner = NFoldPartitioner(cvtype=1)

In [ ]:
clfmode = 'LinearCSVMC'

In [ ]:
# Choose right classifier
if clfmode == 'LinearCSVMC':
    clf = LinearCSVMC()

elif clfmode == 'LinearNuSVMC':
    clf = LinearNuSVMC()

elif clfmode == 'SMLR':
    smlr_lm = 0.1
    clf = SMLR(lm=smlr_lm)  # [1e-10,inf]

In [ ]:
cv = CrossValidation(clf, partitioner,
                     errorfx=mean_match_accuracy,
                     enable_ca=['stats'])

In [ ]:
sphere_radius = 2
nth_element = 100
cores2use = 8

In [ ]:
sl = sphere_searchlight(cv,
                        radius=sphere_radius,
                        center_ids=range(0,
                                         ds.shape[1],
                                         nth_element),
                        space='voxel_indices',
                        results_fx=fill_in_scattered_results,
                        postproc=mean_sample(),
                        enable_ca=['calling_time', 'roi_feature_ids'])

In [ ]:
sl.nproc = cores2use

In [ ]:
# train classifier on original and permutated dataset
sl_map = sl(ds)
print('orig done after %s' % (
    time.strftime(
        '%H:%M:%S',
        time.gmtime(round(sl.ca.calling_time)))))

In [ ]:
# Calculate Classifier Specific outputs
accuracies = sl_map.samples[0]
mean_accuracy = accuracies.mean()
std_accuracy = accuracies.std()
chance_level = 1.0 / len(ds.sa.get(space).unique)

threshold_above_average = lambda x: chance_level + x * std_accuracy
spheres_above_average = lambda x: np.sum(
    accuracies >= threshold_above_average(x))
percent_above_average = lambda x: np.mean(
    accuracies >= threshold_above_average(x)) * 100

# Save searchlight accuracy map to NIfTI file
niftiresults = map2nifti(ds, data=sl_map.S, imghdr=ds.a.imghdr)
niftiresults.to_filename(opj(outpath, 'nifti_%s.nii.gz' % identifier))

# Write report to file
csvfile = opj(outpath, 'report_%s.rst' % identifier)
with open(csvfile, 'w') as f:
    f.write('Subject          : {0}\n'.format(sub))
    f.write('Classifier       : {0}\n'.format(clfmode))
    f.write('Category         : {0}\n'.format(category))
    f.write('Classes          : {0}\n'.format(classes))
    f.write('Comparison       : {0}\n'.format(comparison))
    f.write('Sphere Radius    : {0}\n'.format(sphere_radius))
    f.write('N-th Element     : {0}\n'.format(nth_element))
    f.write('Wall Time        : {0}\n'.format(
        time.strftime('%H:%M:%S', time.gmtime(round(sl.ca.calling_time)))))
    f.write('Samples          : {0}\n'.format(ds.S.shape[0]))
    f.write('Features         : {0}\n'.format(ds.S.shape[1]))
    f.write('Volume Dimension : {0}\n'.format(str(ds.a.voxel_dim)))
    f.write('Voxel  Dimension : {0}\n'.format(str(ds.a.voxel_eldim)))
    f.write('endfix           : {0}\n'.format(endfix))
    f.write('CPU              : {0}\n'.format(cores2use))

    f.write('\nChance Level     : {0}\n'.format(round(chance_level, 5)))
    f.write('Accuracy (mean)  : {0:5}\n'.format(round(mean_accuracy, 5)))
    f.write('Accuracy (std)   : {0:5}\n\n'.format(round(std_accuracy, 5)))

    f.write('%Sphere > +2STD  : {0:5}%\n'.format(round(percent_above_average(2), 3)))
    f.write('%Sphere > +3STD  : {0:5}%\n'.format(round(percent_above_average(3), 3)))
    f.write('%Sphere > +4STD  : {0:5}%\n'.format(round(percent_above_average(4), 3)))
    f.write('vSphere > +2STD  : {0:5}\n'.format(spheres_above_average(2)))
    f.write('vSphere > +3STD  : {0:5}\n'.format(spheres_above_average(3)))
    f.write('vSphere > +4STD  : {0:5}\n'.format(spheres_above_average(4)))

    f.write('\n\nDataset Summary:\n')
    f.write('****************\n')
    f.write('%s' % ds.summary())
    f.write('%s' % ds.summary)

In [ ]:
ds

In [ ]:
dataset

In [ ]:
dataset.sa.targets

In [ ]:
ds.sa.targets

### Specify Classifier

But now for the interesting part: Next we define the measure that shall be computed for each sphere. Theoretically, this can be anything, but here we choose to compute a full leave-one-out cross-validation using a linear Nu-SVM classifier.

In [ ]:
# choose classifier
clf = LinearNuSVMC()

# setup measure to be computed by Searchlight
# cross-validated mean transfer using an N-fold dataset splitter
cv = CrossValidation(clf, NFoldPartitioner())

In this example, we do not want to compute full-brain accuracy maps, but instead limit ourselves to a specific subset of voxels. We’ll select all voxel that have a non-zero z-stats value in the localizer mask we loaded above, as center coordinates for a searchlight sphere. These spheres will still include voxels that did not pass the threshold. the localizer merely define the location of all to be processed spheres.

In [ ]:
# get ids of features that have a nonzero value
center_ids = ds.fa.voxel_indices

Finally, we can run the searchlight. We’ll perform the analysis for three different radii, each time computing an error for each sphere. To achieve this, we simply use the [sphere_searchlight()](http://www.pymvpa.org/generated/mvpa2.measures.searchlight.sphere_searchlight.html#mvpa2.measures.searchlight.sphere_searchlight) class, which takes any [processing object](http://www.pymvpa.org/glossary.html#term-processing-object) and a radius as arguments. The [processing object](http://www.pymvpa.org/glossary.html#term-processing-object) has to compute the intended measure, when called with a dataset. The [sphere_searchlight()](http://www.pymvpa.org/generated/mvpa2.measures.searchlight.sphere_searchlight.html#mvpa2.measures.searchlight.sphere_searchlight) object will do nothing more than generate small datasets for each sphere, feeding them to the processing object, and storing the result.

In [ ]:
# setup plotting parameters (not essential for the analysis itself)
plot_args = {
    'background' : os.path.join(datapath, 'haxby2001', 'sub001', 'anatomy', 'highres001.nii.gz'),
    'background_mask' : os.path.join(datapath, 'haxby2001', 'sub001', 'masks', 'orig', 'brain.nii.gz'),
    'overlay_mask' : os.path.join(datapath, 'haxby2001', 'sub001', 'masks', 'orig', 'vt.nii.gz'),
    'do_stretch_colors' : False,
    'cmap_bg' : 'gray',
    'cmap_overlay' : 'autumn', # YlOrRd_r # pl.cm.autumn
    'interactive' : cfg.getboolean('examples', 'interactive', True),
    }

### Run Searchlight

In [ ]:
# Chose Sphere radius
radius = 0

# tell which one we are doing
print("Running searchlight with radius: %i ..." % (radius))

Here we actually setup the spherical searchlight by configuring the radius, and our selection of sphere center coordinates. Moreover, via the **space** argument we can instruct the searchlight which feature attribute shall be used to determine the voxel neighborhood. By default, fmri_dataset() creates a corresponding attribute called **voxel_indices**. Using the **mapper** argument it is possible to post-process the results computed for each sphere. Cross-validation will compute an error value per each fold, but here we are only interested in the mean error across all folds. Finally, on multi-core machines **nproc** can be used to enabled parallelization by setting it to the number of processes utilized by the searchlight (default value of **nproc`=`None** utilizes all available local cores).

In [ ]:
sl = sphere_searchlight(cv, radius=radius, space='voxel_indices',
                        center_ids=center_ids,
                        postproc=mean_sample())

Since we care about efficiency, we are stripping all attributes from the dataset that are not required for the searchlight analysis. This will offers some speedup, since it reduces the time that is spent on dataset slicing.

In [ ]:
ds = dataset.copy(deep=False,
                  sa=['targets', 'chunks'],
                  fa=['voxel_indices'],
                  a=['mapper'])

Finally, we actually run the analysis. The result is returned as a dataset. For the upcoming plots, we are transforming the returned error maps into accuracies.

In [ ]:
# Runs the serachlight
sl_map = sl(ds)

# Changes output from error maps to accuracy maps
sl_map.samples *= -1
sl_map.samples += 1

### Investigat the results

The result dataset is fully aware of the original dataspace. Using this information we can map the 1D accuracy maps back into “brain-space” (using NIfTI image header information from the original input timeseries.

In [ ]:
niftiresults = map2nifti(sl_map, imghdr=dataset.a.imghdr)

PyMVPA comes with a convenient plotting function to visualize the searchlight maps. We are only looking at fMRI slices that are covered by the mask of ventral temproal cortex.

The following figures show the resulting accuracy maps for the slices covered by the ventral temporal cortex mask. Note that each voxel value represents the accuracy of a sphere centered around this voxel.

In [ ]:
print('Searchlight (single element; univariate) accuracy maps '
      'for binary classification house vs. scrambledpix.')
fig = pl.figure(figsize=(12, 8), facecolor='white')
subfig = plot_lightbox(overlay=niftiresults,
                       vlim=(0.0, None), slices=range(23,29),
                       fig=fig, **plot_args);

With radius 0 (only the center voxel is part of the part the sphere) there is a clear distinction between two distributions. The chance distribution, relatively symetric and centered around the expected chance-performance at 50%. The second distribution, presumambly of voxels with univariate signal, is nicely segregated from that. 

### Run searchlight again, but this time with a sphere radius of 3

Increasing the searchlight size significantly blurrs the accuracy map, but also lead to an increase in classification accuracy. So let's try the searchlight again with a sphere radius of 3.

In [ ]:
radius = 3
sl = sphere_searchlight(cv, radius=radius, space='voxel_indices',
                        center_ids=center_ids, postproc=mean_sample())
sl_map = sl(ds)
sl_map.samples *= -1
sl_map.samples += 1
niftiresults = map2nifti(sl_map, imghdr=dataset.a.imghdr)

In [ ]:
print('Searchlight (radius 3 elements; 123 voxels) accuracy maps for '
      'binary classification house vs. scrambledpix.')
fig = pl.figure(figsize=(12, 8), facecolor='white')
subfig = plot_lightbox(
    overlay=niftiresults, vlim=(0.0, None),
    slices=range(23,29), fig=fig, **plot_args)